In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

In [19]:
# Création d'un jeu de données personnalisé
from CLASSES_DEEP_V2 import VideosDataset

In [20]:
image_dir = 'train/images2'
annotation_dir = 'train/labels2'

annotations = []
for filename in os.listdir(annotation_dir):
    if filename.endswith('.csv'):
        filepath = os.path.join(annotation_dir, filename)
        try:
            df = pd.read_csv(filepath, header=None)
            if not df.empty:
                annotations.extend(df.iloc[:, 4].values)
        except Exception as e:
            pass

label_encoder = LabelEncoder()
label_encoder.fit(annotations)

LabelEncoder()

In [21]:
def collate_fn(batch):
    images = []
    labels = []

    for item in batch:
        try:
            for roi, label in item:
                images.append(roi)
                labels.append(label)
        except Exception as e:
            images.append(item[0])
            labels.append(item[1])

    images = torch.stack(images, dim=0)
    labels = torch.tensor(labels)

    return images, labels

In [22]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = VideosDataset(image_dir='train/images2',
                              annotation_dir='train/labels2',
                              label_encoder=label_encoder,
                              transform=transform)

val_dataset = VideosDataset(image_dir='val/images',
                            annotation_dir='val/labels',
                            label_encoder=label_encoder,
                            transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [23]:
# Définition du modèle
from classes import SimpleCNN

num_classes = len(label_encoder.classes_)
model = SimpleCNN(num_classes)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
num_epochs = 10

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch+1}/{num_epochs}')
    model.train()
    running_loss = 0.0
    i = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f'Batch {i+1}/{len(train_loader)} done. Loss: {loss.item()}')
        i += 1

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%')

AttributeError: 'SimpleCNN' object has no attribute 'fit'

In [ ]:
 # Évaluation finale sur le jeu de validation
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Final Validation Loss: {val_loss/len(val_loader)}, Final Accuracy: {100 * correct / total}%')

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np

model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt=".2f", cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')

In [ ]:
import pickle
with open('simple_cnn.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
with open('cnn_label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)